In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

** Dog Breed Recognition**

This notebook demonstrates how to use a Convolutional Neural Network (CNN) to recognize the breed of a dog from an image.
We will train a model on a labeled dataset of dog images and use it to predict the breed of a new image.

1️. Importing Libraries

We import libraries for data processing, visualization, and building a CNN model.

2️. Loading the Data

The CSV contains image IDs and their corresponding dog breeds.
We append .jpg to each ID so the images can be loaded.

3️. Preparing the Images

Images are normalized (divided by 255) and split into training and validation sets.

4️. Building the CNN Model

The model has four layers, MaxPooling layers, Flatten, and Dense layers.
The output layer uses softmax for multi-class classification of dog breeds.

5️. Compiling and Training

We use Adam optimizer and categorical_crossentropy loss because this is a multi-class classification problem.

6️. Visualizing Accuracy and Loss

We monitor how the model is learning and check for overfitting.

7️. Predicting the Breed of a New Image

The model predicts the breed of a new dog image and shows the confidence percentage.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

labels = pd.read_csv('/kaggle/input/dog-breed-identification-data/labels.csv')
labels['id'] = labels['id'] + '.jpg'
labels.head()

from tensorflow.keras.preprocessing.image import ImageDataGenerator
img_size = 224
batch_size = 32
EPOCHS = 25

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=labels,
    directory='/kaggle/input/dog-breed-identification-data/train',
    x_col='id',
    y_col='breed',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_dataframe(
    dataframe=labels,
    directory='/kaggle/input/dog-breed-identification-data/train',
    x_col='id',
    y_col='breed',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

num_classes = len(train_generator.class_indices)
inputs = Input(shape=(img_size, img_size, 3))
x = Conv2D(32, (3,3), activation='relu')(inputs)
x = MaxPooling2D(2,2)(x)

x = Conv2D(64, (3,3), activation='relu')(x)
x = MaxPooling2D(2,2)(x)

x = Conv2D(128, (3,3), activation='relu')(x)
x = MaxPooling2D(2,2)(x)

x = Conv2D(224, (3,3), activation='relu')(x)
x = MaxPooling2D(2,2)(x)

x = Flatten()(x)
x = Dense(224, activation='relu')(x)
x = Dropout(0.5)(x)

outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

model.summary()

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS
)
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Loss')
plt.legend()

plt.show()








In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

img_path = '/kaggle/input/my-dog-jpg/2.jpg'  

class_names = list(train_generator.class_indices.keys())
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)


img_array = img_array / 255.0


img_array = np.expand_dims(img_array, axis=0)


predictions = model.predict(img_array)

predicted_index = np.argmax(predictions)
predicted_breed = class_names[predicted_index]
confidence = np.max(predictions)

plt.imshow(img)
plt.axis('off')
plt.title(f"Predicted breed: {predicted_breed} ({confidence*100:.2f}%)")
plt.show()

top_5 = np.argsort(predictions[0])[-5:][::-1]

print("Top-5 predictions:")
for i in top_5:
    print(f"{class_names[i]} ({predictions[0][i]*100:.2f}%)")
